num of Float : about 50M

In [35]:
from ellzaf_ml.models import GhostFaceNetsV2
import torch
from fvcore.nn import FlopCountAnalysis
model = GhostFaceNetsV2(image_size=112, width=1.3, dropout=0.)
model.eval()
input_tensor = torch.randn(1, 3, 112, 112) 
flops = FlopCountAnalysis(model, input_tensor)
print(f"Total FLOPS: {flops.total()}")
# print(model.output)

In [34]:
from ellzaf_ml.models.mobilefacenet import MobileFaceNet
import torch
from fvcore.nn import FlopCountAnalysis
model = MobileFaceNet(512)
model.eval()
input_tensor = torch.randn(1, 3, 112, 112) 
flops = FlopCountAnalysis(model, input_tensor)
print(f"Total FLOPS: {flops.total()}")
# print(model.output)

In [1]:
import torch
from ellzaf_ml.models import GhostFaceNetsV2
from torchsummary import summary
IMAGE_SIZE = 112
import time
#return embedding
model = GhostFaceNetsV2(image_size=IMAGE_SIZE, width=1.3, dropout=0.2)
img = torch.randn(1, 3, IMAGE_SIZE, IMAGE_SIZE)
model =model.cuda()
img =img.cuda()
model.eval()

#return classification
# model = GhostFaceNetsV2(image_size=IMAGE_SIZE, num_classes=10, width=1, dropout=0.)
# img = torch.randn(3, 3, IMAGE_SIZE, IMAGE_SIZE)



In [26]:
x=time.time()
size = model(img)
y=time.time()
print(size.size())
print(y-x)

In [9]:
input_data = torch.randn(3, 3, IMAGE_SIZE, IMAGE_SIZE)
print(input_data.dtype)
input_data = input_data.half()
print(input_data.dtype)

In [4]:
from losses import CombinedMarginLoss
from configs.ghostfacenets import config as cfg
import torch
margin_loss = CombinedMarginLoss(
64,
cfg.margin_list[0],
cfg.margin_list[1],
cfg.margin_list[2],
cfg.interclass_filtering_threshold
)
logits = torch.randn(5, 3)  # Logits ngẫu nhiên
labels = torch.randint(0, 3, (5,))
print(logits)
print(labels)
loss = margin_loss(logits, labels)
print(loss)

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import math
class MyLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(MyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.bias = Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return F.linear(input, self.weight, self.bias)

# Example usage:
layer = MyLinear(10, 5)
print(layer.weight)
print(layer.bias)


In [23]:
from margin_model import MarginModel
model = MarginModel(512,1000)
img = torch.randn((32,512))
kq = model(img)
print(kq)
print(kq.shape)
opt = torch.optim.SGD(
    params=[{"params": model.parameters()}],
    lr=cfg.lr, momentum=0.9, weight_decay=cfg.weight_decay)

print(model.weight)

In [41]:
from torchsummary import summary
from ellzaf_ml.models.ghostfacenetsv2 import GhostFaceNetsV2
model = GhostFaceNetsV2(image_size=112,num_classes=2,  width=1, dropout=0.)
model=model.cuda()
image = torch.randn([3,3,112,112])
image = image.cuda()
print(image.shape)
kq = model(image)
print(kq)
# summary(model,(3,112,112))

In [52]:
from ellzaf_ml.models.ghostfacenetsv2 import ModifiedGDC
import torch
from torchsummary import summary
modelGDC = ModifiedGDC(image_size=112, in_chs=960, num_classes=10000, dropout=0.3)
modelGDC =modelGDC.cuda()
# img = torch.randn((3,960,4,4))
# img =img.cuda()
# modelGDC(img)
summary(modelGDC, (960,4,4)) 

In [49]:
import torch
import torch.nn as nn
from torchsummary import summary

class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        
        self.conv_dw = nn.Conv2d(960, 960, kernel_size=(4, 4), stride=(1, 1), groups=960, bias=False)
        self.bn1 = nn.BatchNorm2d(960, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        self.dropout = nn.Dropout(p=0.0, inplace=False)
        self.conv = nn.Conv2d(960, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn2 = nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.linear = nn.Linear(512, 10, bias=True)

    def forward(self, x):
        x = self.conv_dw(x)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.conv(x)
        x = x.view(x.size(0), -1) # Flatten
        x = self.bn2(x)
        x = self.linear(x)
        return x

# Khởi tạo mô hình
model = CustomModel()

# Chuyển mô hình lên GPU nếu có sử dụng CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Sử dụng summary để hiển thị thông tin mô hình và số lượng tham số
summary(model, (960, 16, 16))  # Đưa vào kích thước đầu vào (channels, height, width)

tại sao có sự khác biệt giữa tham số 2 mô hình pytorch và keras được implement trong paper

ta có model của keras có tổng tham số là 6880288 (giống trong bài báo)  
ở đây có 2 lý do, 1 số lượng tham số của Batchnormalization của Keras gấp  đôi Pytorch =84568  
Thứ 2 là số lượng tham số của hàm PReLu trong pytorch chỉ là 1 và trong keras thì tùy từng trường hợp có thể khác rất nhiêu  
Tôi đã đếm được model Pytorch sử dụng 41 hàm PRelu và Keras có 11388 tham số của PRelu  
Ta có tham số cơ bản của pytorch model là 6,784,373 tham số  
Khi chúng ta tính toán và trừ đi thì   
6880288 - 6784373 + 41 - 84568 -11388 = 0  
đã giải thích được số lượng tham số của 2 phiên bản tại sao khác nhau

In [ ]:
model=model.cuda()
summary(model,(3,112,112))
import torch.nn as nn
def count_batchnorm_params(model):
    total_params = 0
    prelu_params = 0
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d) or isinstance(module,nn.BatchNorm1d) :
            for param in module.parameters():
                total_params += param.numel()
        if isinstance(module,nn.PReLU):
            prelu_params +=1
    print("prelu_params",prelu_params)
    return total_params

# Gọi hàm và in kết quả
num_bn_params = count_batchnorm_params(model)
print(f"Số lượng tham số của tất cả các lớp BatchNorm2d trong mô hình: {num_bn_params}")

In [27]:
import torch
print(*torch.__config__.show().split("\n"), sep="\n")

In [42]:
from ellzaf_ml.models.mobilefacenet import MobileFaceNet
net = MobileFaceNet(512)
net.cpu()
net.eval()

In [12]:
import torch
from ellzaf_ml.models import GhostFaceNetsV2
from torchsummary import summary
IMAGE_SIZE = 112
import time
#return embedding
model = GhostFaceNetsV2(image_size=IMAGE_SIZE, width=1.3, dropout=0.2)
img = torch.randn(32, 3, IMAGE_SIZE, IMAGE_SIZE)
model =model.cuda()
img =img.cuda()
model.eval()

# img = img.to_mkldnn()
# print(img.shape)
# x =time.time()
# net(img)
# y= time.time()
# print(y-x)
model.cuda()
x =time.time()
model(img)
y= time.time()
print(y-x)



In [29]:
import torch
from ellzaf_ml.models import GhostFaceNetsV2
from torchsummary import summary
IMAGE_SIZE = 112
import time
#return embedding
model = GhostFaceNetsV2(image_size=IMAGE_SIZE, width=1.3, dropout=0.2)
img = torch.randn(5, 3, IMAGE_SIZE, IMAGE_SIZE)
model = model.cuda()
img = img.cuda()
model.eval()
# model(img)
#return classification
# model = GhostFaceNetsV2(image_size=IMAGE_SIZE, num_classes=10, width=1, dropout=0.)
# img = torch.randn(3, 3, IMAGE_SIZE, IMAGE_SIZE)
x=time.time()
size = model(img)
y=time.time()
print(y-x)
print(size.shape)

load model Mobilefacenet to test

In [1]:
from ellzaf_ml.models.mobilefacenet import MobileFaceNet
import torch
device = "cuda:0"
detect_model = MobileFaceNet(512).to(device)  # embeding size is 512 (feature vector)
detect_model.load_state_dict(torch.load('Weights/MobileFace_Net', map_location=lambda storage, loc: storage))

In [2]:
from torchsummary import summary
summary(detect_model, (3,112,112))

Face alignment test

In [13]:
from MTCNN import create_mtcnn_net_facebank
import cv2
import numpy as np
image_path = "image/2990.jpg"

img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
bboxes, landmarks = create_mtcnn_net_facebank(img, 20, "cpu",
                                                 p_model_path='Weights/pnet_Weights',
                                                 r_model_path='Weights/rnet_Weights',
                                                 o_model_path='Weights/onet_Weights')
print(landmarks)



In [14]:
import numpy as np
from scipy.spatial import distance

# Điểm đặc trưng đã phát hiện, dạng 1D
detected_points = landmarks

# Tách tọa độ x và y
x_coords = detected_points[0, :5]
y_coords = detected_points[0, 5:]

# Kết hợp x và y thành mảng (5, 2)
detected_points_reshaped = np.stack((x_coords, y_coords), axis=1)
print(detected_points_reshaped)
# Các bộ tọa độ chuẩn đã cho
coord5point_1 = np.array([[38.29459953, 51.69630051],
                          [73.53179932, 51.50139999],
                          [56.02519989, 71.73660278],
                          [41.54930115, 92.3655014],
                          [70.72990036, 92.20410156]], dtype=np.float32)

coord5point_2 = np.array([[30.29459953, 51.69630051],
                          [65.53179932, 51.50139999],
                          [48.02519989, 71.73660278],
                          [33.54930115, 92.3655014],
                          [62.72990036, 92.20410156]], dtype=np.float32)
print(coord5point_1)

# Tính tổng khoảng cách Euclidean giữa các điểm đặc trưng đã phát hiện và từng bộ tọa độ chuẩn
distance_1 = np.sum([distance.euclidean(detected_points_reshaped[i], coord5point_1[i]) for i in range(5)])
distance_2 = np.sum([distance.euclidean(detected_points_reshaped[i], coord5point_2[i]) for i in range(5)])

# In ra tổng khoảng cách để kiểm tra
print("Tổng khoảng cách với coord5point_1:", distance_1)
print("Tổng khoảng cách với coord5point_2:", distance_2)

# So sánh tổng khoảng cách để xác định loại căn chỉnh
if distance_1 < distance_2:
    print("Hình ảnh thuộc loại căn chỉnh thứ nhất (coord5point_1).")
else:
    print("Hình ảnh thuộc loại căn chỉnh thứ hai (coord5point_2).")


validation on lfw mobile facenet

In [ ]:

ctx = mx.gpu(args.gpu)